In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 5.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers, initializers
from tensorflow.keras.layers import Concatenate, Input, Embedding, LSTM, Attention, Dense, Dropout, Bidirectional, LSTM, Flatten, Conv1D, GlobalMaxPool1D, MaxPool1D, Reshape, GlobalAveragePooling1D, UpSampling1D, Layer

from subprocess import check_output

import tensorflow
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import datetime
import sys
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)
nltk.download('punkt')

tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# for mounting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read data train and test
train_df = pd.read_csv('/content/drive/fasttext/train9000.csv', sep=',', header=0)
test_df = pd.read_csv('/content/drive/fasttext/test900.csv', sep=',', header=0)

print("num train: ", train_df.shape[0])
print("num test: ", test_df.shape[0])

num train:  9737
num test:  900


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factoryStopword = StopWordRemoverFactory()
stopword = factoryStopword.create_stop_word_remover()
stop_factory = StopWordRemoverFactory().get_stop_words()

def get_stopwords():
  docs = pd.read_csv('/content/drive/fasttext/stopwords-id.txt', header = None, names= ['stopword']) # read stopwords
  docs = docs['stopword'].tolist()
  return docs

stop_words = get_stopwords()

# Merge stopword
data = set(stop_factory + stop_words)

#remove stopword
def removeStopword(str):
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in data]
    return ' '.join(filtered_sentence) 

def processing(str):
  str = str.lower()
  #str = removeStopword(str)
  return str

In [ ]:
# lowering text
train_df['text'] = train_df['text'].apply(processing)
test_df['text'] = test_df['text'].apply(processing)

# merge columns
train_df['text'] = train_df[train_df.columns[:2]].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1)

test_df['text'] = test_df[test_df.columns[:2]].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [ ]:
raw_docs_train = train_df['text'].tolist()
raw_docs_test = test_df['text'].tolist() 
num_classes = 3

In [ ]:
# preprocessing data train
print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    processed_docs_train.append(doc)

pre-processing train data...


100%|██████████| 9737/9737 [00:00<00:00, 51670.22it/s]


In [ ]:
np.savetxt('trainheadline.txt', processed_docs_train, delimiter=' ', fmt="%s")
! wc trainheadline.txt
! head -n 9737 trainheadline.txt > headlineImbalance.train

  9737 109928 842606 trainheadline.txt


In [ ]:
# preprocessing data test
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    processed_docs_test.append(doc)

100%|██████████| 900/900 [00:00<00:00, 19236.10it/s]


In [ ]:
np.savetxt('testheadline.txt', processed_docs_test, delimiter=' ', fmt="%s")
! wc testheadline.txt
! head -n 900 testheadline.txt > headline.test

  900  9250 72192 testheadline.txt


In [ ]:
# downloading fasttext facebook
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

--2022-01-05 07:02:53--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2022-01-05 07:02:53--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 13.112.159.149
Connecting to codeload.github.com (codeload.github.com)|13.112.159.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  11.4MB/s    in 0.4s    

2022-01-05 07:02:54 (11.4 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastTe

In [ ]:
# doing fasttext

import fasttext

training_data_path ='/content/headlineImbalance.train' 
validation_data_path ='/content/headline.test'
model_path ='/content/'
model_name="model-headline"
vec_docs = "/content/drive/fasttext/wiki.id.vec" # using fasttext vector indonesia 300

def train():
    print('Training start')
    try:
        hyper_params = {"lr": 0.008,
                        "epoch": 10,
                        "wordNgrams": 4,
                        "dim": 300,
                        "pretrainedVectors": vec_docs
                        }     
                               
        print(str(datetime.datetime.now()) + ' START=>' + str(hyper_params) )

        # Train the model.
        global model
        model = fasttext.train_supervised(input=training_data_path, **hyper_params)
        print("Model trained with the hyperparameter \n {}".format(hyper_params))

        # CHECK PERFORMANCE
        print(str(datetime.datetime.now()) + 'Training complete.' + str(hyper_params) )
        
        result = model.test(training_data_path)
        validation = model.test(validation_data_path)
        
        # DISPLAY ACCURACY OF TRAINED MODEL
        text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
        print(text_line)
        
        #quantize a model to reduce the memory usage
        model.quantize(input=training_data_path, qnorm=True, retrain=True, cutoff=100000)
        print("Model is quantized!!")
        model.save_model(os.path.join(model_path,model_name + ".ftz"))                
    
        #TESTING        
        print(model.predict(['Kasus Mutilasi di Sumbawa: Ditemukan di Dalam Kulkas, 11 Saksi Diperiksa Polisi Termasuk Suami'],k=3)) #neg
        print(model.predict(['Armuji Siap Mundur dari DPRD Jatim Jika Terpilih Sebagai Calon Wali Kota Surabaya'],k=1)) #pos
        print(model.predict(['Jadi Istri Pejabat, Arumi Bachsin Makan Bareng Chef Juna di Hari Ikan Nasional'], k=1)) #neu
        
        
    except Exception as e:
        print('Exception during training: ' + str(e) )


# Train your model.
train()

Training start
2022-01-05 07:04:04.224976 START=>{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
Model trained with the hyperparameter 
 {'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
2022-01-05 07:04:54.991210Training complete.{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'},accuracy:0.6550272157748793,validation:0.5622222222222222

Model is quantized!!
([['__label__NEUTRAL', '__label__PO

In [ ]:
# y_true
fmov=open('/content/testheadline.txt','r')
sentence_list=[]
label_list=[]
for f in fmov.readlines():
    lineinfo=f.split(' ')
    sentenceline=".".join(lineinfo[0:-1])
    sentence_list.append(sentenceline)
    label_list.append(str(lineinfo[0]).replace('\n',''))
    y_true = label_list

In [ ]:
# y_pred
y_pred = model.predict(processed_docs_test, k=1)
y_pred2 = list(y_pred)
y_pred2 = y_pred2.pop(0)
out = np.concatenate(y_pred2).ravel().tolist()
y_pred = out

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print("confusion matrix: \n", confusion_matrix(y_true, y_pred))

# Accuracy
print("accuracy: ", accuracy_score(y_true, y_pred))

# Recall
from sklearn.metrics import recall_score
print("recall: \n", recall_score(y_true, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("precision: \n", precision_score(y_true, y_pred, average=None))

confusion matrix: 
 [[194  52  54]
 [ 65 137  98]
 [ 46  96 158]]
accuracy:  0.5433333333333333
recall: 
 [0.64666667 0.45666667 0.52666667]
precision: 
 [0.63606557 0.48070175 0.50967742]
